# Extraction

Outra aplicação comum para funções é a relização de extração de conteúdos do texto informado. Isso facilita o parseamento de informações em grande escala, em que podemos utilizar scripts para limpar textos informados, mantendo apenas o necessário da informação que o usuário necessita. Vamos começar com um exemplo simples, extraindo a informação de datas e acontecimentos de um texto.

## Extraindo datas e acontecimentos

Digamos que temos um texto como o a seguir e queremos extrair datas e acontecimentos que aparecerem no texto:


In [4]:
texto = '''A Apple foi fundada em 1 de abril de 1976 por Steve Wozniak, Steve Jobs e Ronald Wayne 
com o nome de Apple Computers, na Califórnia. O nome foi escolhido por Jobs após a visita do pomar 
de maçãs da fazenda de Robert Friedland, também pelo fato do nome soar bem e ficar antes da Atari 
nas listas telefônicas.

O primeiro protótipo da empresa foi o Apple I que foi demonstrado na Homebrew Computer Club em 1975, 
as vendas começaram em julho de 1976 com o preço de US$ 666,66, aproximadamente 200 unidades foram 
vendidas,[21] em 1977 a empresa conseguiu o aporte de Mike Markkula e um empréstimo do Bank of America.'''

In [2]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function

class Acontecimento(BaseModel):
    '''Informação sobre um acontecimento'''
    data: str = Field(description='Data do acontecimento no formato YYYY-MM-DD')
    acontecimento: str = Field(description='Acontecimento ecxtraído do texto')

class ListaAcontecimentos(BaseModel):
    """Acontecimentos para extração"""
    acontecimentos: List[Acontecimento] = Field(description='Lista de acontecimentos presentes no texto informado')

tool_acontecimentos = convert_to_openai_function(ListaAcontecimentos)
tool_acontecimentos

{'name': 'ListaAcontecimentos',
 'description': 'Acontecimentos para extração',
 'parameters': {'type': 'object',
  'properties': {'acontecimentos': {'description': 'Lista de acontecimentos presentes no texto informado',
    'type': 'array',
    'items': {'description': 'Informação sobre um acontecimento',
     'type': 'object',
     'properties': {'data': {'description': 'Data do acontecimento no formato YYYY-MM-DD',
       'type': 'string'},
      'acontecimento': {'description': 'Acontecimento ecxtraído do texto',
       'type': 'string'}},
     'required': ['data', 'acontecimento']}}},
  'required': ['acontecimentos']}}

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extraia as frases de acontecimentos. Elas devem ser extraídas integralmente'),
    ('user', '{input}')
])
chat = ChatOpenAI()

chain = (prompt 
         | chat.bind(functions=[tool_acontecimentos], function_call={'name': 'ListaAcontecimentos'}))


In [5]:
chain.invoke({'input': texto})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"acontecimentos":[{"data":"1976-04-01","acontecimento":"A Apple foi fundada por Steve Wozniak, Steve Jobs e Ronald Wayne."},{"data":"1975-07","acontecimento":"Início das vendas do Apple I com o preço de US$ 666,66."},{"data":"1977","acontecimento":"A Apple recebeu aporte de Mike Markkula e um empréstimo do Bank of America."}]}', 'name': 'ListaAcontecimentos'}}, response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 325, 'total_tokens': 426}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ad3b401a-654f-4d2a-b120-630b0d964429-0')

In [6]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (prompt 
         | chat.bind(functions=[tool_acontecimentos], function_call={'name': 'ListaAcontecimentos'})
         | JsonOutputFunctionsParser())

chain.invoke({'input': texto})

{'acontecimentos': [{'data': '1976-04-01',
   'acontecimento': 'A Apple foi fundada em 1 de abril de 1976 por Steve Wozniak, Steve Jobs e Ronald Wayne com o nome de Apple Computers, na Califórnia.'},
  {'data': '1975-07',
   'acontecimento': 'As vendas do Apple I começaram em julho de 1976 com o preço de US$ 666,66, aproximadamente 200 unidades foram vendidas.'},
  {'data': '1977',
   'acontecimento': 'Em 1977 a empresa conseguiu o aporte de Mike Markkula e um empréstimo do Bank of America.'}]}

In [7]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (prompt 
         | chat.bind(functions=[tool_acontecimentos], function_call={'name': 'ListaAcontecimentos'})
         | JsonKeyOutputFunctionsParser(key_name='acontecimentos'))

chain.invoke({'input': texto})

[{'data': '1976-04-01',
  'acontecimento': 'Apple foi fundada por Steve Wozniak, Steve Jobs e Ronald Wayne.'},
 {'data': '1975',
  'acontecimento': 'Demonstração do primeiro protótipo da empresa, o Apple I, na Homebrew Computer Club.'},
 {'data': '1976-07',
  'acontecimento': 'Início das vendas do Apple I com o preço de US$ 666,66.'},
 {'data': '1977',
  'acontecimento': 'Empresa recebeu aporte de Mike Markkula e empréstimo do Bank of America.'}]

## Extraindo informações da web

A aplicação de extração pode ser muito utilizanda quando combinada com técnicas de WebScraping. Em geral, em WebScraping estamos buscando informações em páginas web. Em sua grande maioria, essas informações virão comletamente desformatadas e em html, o que dificulta a utilização da informação. Podemos criar aplicações utilizando as técnicas que aprendemos para conseguir as informações específicas que precisamos das páginas que estamos analisando.

Vamos dar um exemplo analisando a página de blog da Asimov e tentando extrair todas os posts contidos na página.
https://hub.asimov.academy/blog/


In [8]:
from langchain_community.document_loaders.web_base import WebBaseLoader

loader = WebBaseLoader('https://hub.asimov.academy/blog/')
page = loader.load()
page

[Document(page_content='\n\n\n\n\n\n\nBlog | Asimov Academy\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n \n\n\n\nHome\n\n\nTrilhas \n\n\n\nTrilha Aplicações IA com Python\nTrilha Dashboards com Python\nTrilha Python Office\nTrilha Visão Computacional\nTrilha Data Science e Machine Learning\nTrilha Trading Quantitativo\n\n\n\nCursos\n\n\nProjetos\n\n\nEntrar\n\n\n\n\nMatricule-se\n\n\n\n\n\n Blog da Asimov \n\n\n \n\n \n\n \n\n \n\n\n\n\n\n\n\n Python ou Power BI: qual ferramenta é melhor para criar dashboards? \n Renata Lopes• 23-05-24 \n\n\n\n\n\n Try e Except em Python - Entenda como lidar com erros \n Adriano Soares• 22-05-24 \n\n\n\n\n\n Print em Python: entenda o que é a função print e onde usá-la \n Juliano Faccioni• 21-05-24 \n\n\n\n\n\n Exemplos de programas em Python - Aprenda fazendo \n Adriano Soares• 16-05-24 \n\n\n\n\n\n Qual linguagem de programação 

In [9]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function

class BlogPost(BaseModel):
    '''Informações sobre um post de blog'''
    titulo: str = Field(description='O título do post de blog')
    autor: str = Field(description='O autor do post de blog')
    data: str = Field(description='A data de publicação do post de blog')

class BlogSite(BaseModel):
    '''Lista de blog posts de um site'''
    posts: List[BlogPost] = Field(description='Lista de posts de blog do site')

tool_blog = convert_to_openai_function(BlogSite)
tool_blog

{'name': 'BlogSite',
 'description': 'Lista de blog posts de um site',
 'parameters': {'type': 'object',
  'properties': {'posts': {'description': 'Lista de posts de blog do site',
    'type': 'array',
    'items': {'description': 'Informações sobre um post de blog',
     'type': 'object',
     'properties': {'titulo': {'description': 'O título do post de blog',
       'type': 'string'},
      'autor': {'description': 'O autor do post de blog', 'type': 'string'},
      'data': {'description': 'A data de publicação do post de blog',
       'type': 'string'}},
     'required': ['titulo', 'autor', 'data']}}},
  'required': ['posts']}}

In [10]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extraia da página todos os posts de blog com autor e data de publicação'),
    ('user', '{input}')
])
chat = ChatOpenAI()
chain = (prompt 
         | chat.bind(functions=[tool_blog], function_call={'name': 'BlogSite'})
         | JsonKeyOutputFunctionsParser(key_name='posts'))


In [11]:
chain.invoke({'input': page})

[{'titulo': 'Python ou Power BI: qual ferramenta é melhor para criar dashboards?',
  'autor': 'Renata Lopes',
  'data': '23-05-24'},
 {'titulo': 'Try e Except em Python - Entenda como lidar com erros',
  'autor': 'Adriano Soares',
  'data': '22-05-24'},
 {'titulo': 'Print em Python: entenda o que é a função print e onde usá-la',
  'autor': 'Juliano Faccioni',
  'data': '21-05-24'},
 {'titulo': 'Exemplos de programas em Python - Aprenda fazendo',
  'autor': 'Adriano Soares',
  'data': '16-05-24'},
 {'titulo': 'Qual linguagem de programação aprender? Guia completo para iniciantes',
  'autor': 'Renata Lopes',
  'data': '15-05-24'},
 {'titulo': 'Lambda Python: o que é uma função lambda (anônima)?',
  'autor': 'Juliano Faccioni',
  'data': '14-05-24'},
 {'titulo': 'Melhor curso de Python: como escolher?',
  'autor': 'Renata Lopes',
  'data': '07-05-24'},
 {'titulo': 'If, elif e else: entendendo as estruturas condicionais em Python',
  'autor': 'Adriano Soares',
  'data': '07-05-24'},
 {'tit